<a href="https://colab.research.google.com/github/chengyang122/Metter.js-and-MediaPipe-Interactive-game/blob/main/Detectron2BarcodeMaskR-CNN50_FPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
import numpy as np
import json
import pandas as pd
import cv2
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from tqdm import tqdm
# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

time: 5.88 ms (started: 2022-05-23 04:38:10 +00:00)


In [ ]:
#please use my shared file, set a shortcut, all image files are in Anaya.zip (Unzipped Files)

img_address_base = '/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)/'
df1 = pd.read_excel('/content/drive/MyDrive/barcode/AnanyaAnnotation_10-09-2017.xlsx')
df2 = pd.read_excel('/content/drive/MyDrive/barcode/Christian_annotations_2017-10-09.xlsx', header=None)
df2.columns = df1.columns
df2['Image Filename'] = df2['Image Filename'].str[:-4]


frames = [df1, df2]
df = pd.concat(frames, ignore_index=True)
img = cv2.imread(img_address_base + df.loc[1][0] + '.jpg')
df['isBarcode'] = 1
df['ymax'] = df[['Y1','Y2', 'Y3', 'Y4']].max(axis=1)
df['ymin'] = df[['Y1','Y2', 'Y3', 'Y4']].min(axis=1)
df['xmax'] = df[['X1','X2','X3','X4']].max(axis=1)
df['xmin'] = df[['X1','X2','X3','X4']].min(axis=1)
df['width'] = img.shape[1] #since all shape are the same. Can use shape of one picture 
df['height'] = img.shape[0] #since all shape are the same. Can use shape of one picture 
df['class'] = 1
df['filename'] = df['Image Filename']+ '.jpg'
df["category_id"] = 0

image_names = df.loc[:, 'Image Filename'].unique()
np.random.seed(seed=0)
np.random.shuffle(image_names)
test = image_names[:20]
validation = image_names[20:30]
train = image_names[30:]

test_df = df[df['Image Filename'].isin(test)]
validation_df = df[df['Image Filename'].isin(validation)]
train_df = df[df['Image Filename'].isin(train)]

img_address_base = '/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)/'
img = cv2.imread(img_address_base + train_df.loc[1][0] + '.jpg')

time: 746 ms (started: 2022-05-23 04:05:33 +00:00)


In [ ]:
#show the shape of the output image 
def annotation(row):
    annotation = {}
    area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
    annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]
    annotation['bbox_mode'] = 1
    annotation["category_id"] = row.category_id
    annotation["segmentation"] = [[row.X1, row.Y1, row.X2, row.Y2, row.X3, row.Y3, row.X4, row.Y4]]
    return annotation

dftrain = train_df
dftest = test_df

annotation_column = []
for row in dftrain.itertuples():
    annotation_column.append(annotation(row))
dftrain['annotation'] = annotation_column
g = dftrain[['Image Filename', 'annotation']].groupby('Image Filename')['annotation'].apply(list).reset_index(name='annotations')
g.columns = ['file_name','annotations']
g['width'] = 2592
g['height'] = 1944
g['image_id'] = np.arange(len(g))
g.index=np.arange(len(g))
g.reset_index(inplace=True)
g.to_json("/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)/train_dataset.json",orient="records")

annotation_column = []
for row in dftest.itertuples():
    annotation_column.append(annotation(row))
dftest['annotation'] = annotation_column
g = dftest[['Image Filename', 'annotation']].groupby('Image Filename')['annotation'].apply(list).reset_index(name='annotations')
g.columns = ['file_name','annotations']
g['width'] = 2592
g['height'] = 1944
g['image_id'] = np.arange(len(g))
g.index=np.arange(len(g))
g.reset_index(inplace=True)
g.to_json("/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)/test_dataset.json",orient="records")



time: 69 ms (started: 2022-05-23 04:05:42 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:

def get_board_dicts_train():#/test_dataset.json #/train_dataset.json
    json_file = "/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)"+ '/train_dataset.json' #Fetch the json file
    with open(json_file) as f:
        dataset_dicts = json.load(f)
    for i in dataset_dicts:
        filename = i["file_name"] 
        i["file_name"] = "/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)"+"/"+filename +'.jpg' 
        for j in i["annotations"]:
            j["bbox_mode"] = BoxMode.XYWH_ABS #Setting the required Box Mode
            j["category_id"] = int(j["category_id"])
    return dataset_dicts

def get_board_dicts_test():#/test_dataset.json #/train_dataset.json
    json_file = "/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)"+ '/test_dataset.json' #Fetch the json file
    with open(json_file) as f:
        dataset_dicts = json.load(f)
    for i in dataset_dicts:
        filename = i["file_name"] 
        i["file_name"] = "/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)"+"/"+filename +'.jpg' 
        for j in i["annotations"]:
            j["bbox_mode"] = BoxMode.XYWH_ABS #Setting the required Box Mode
            j["category_id"] = int(j["category_id"])
    return dataset_dicts
from detectron2.data import DatasetCatalog, MetadataCatalog

DatasetCatalog.clear()
#Registering the Dataset
DatasetCatalog.register("boardetect_train", get_board_dicts_train)
MetadataCatalog.get("boardetect_train").set(thing_classes=["Barcode"])

DatasetCatalog.register("boardetect_test", get_board_dicts_test)
MetadataCatalog.get("boardetect_test").set(thing_classes=["Barcode"])

board_metadata_train = MetadataCatalog.get("boardetect_train")
board_metadata_test = MetadataCatalog.get("boardetect_test")

time: 16.9 ms (started: 2022-05-23 04:06:21 +00:00)


In [ ]:
dataset_dicts = get_board_dicts_train()
#Randomly choosing 3 images from the Set
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    print(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=board_metadata_train)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("boardetect_train",)
cfg.DATASETS.TEST = ("boardetect_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9 # set the testing threshold for this model
cfg.DATASETS.TEST = ("boardetect_test", )
predictor = DefaultPredictor(cfg)

from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_board_dicts_test()
decresult = 0
for d in random.sample(dataset_dicts, 46):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=board_metadata_test, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE   # remove the colors of unsegmented pixels
    )
    decresult = decresult + len(outputs['instances'])

time: 17 s (started: 2022-05-23 02:28:38 +00:00)


In [ ]:
dataset_dicts

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9 # set the testing threshold for this model
cfg.DATASETS.TEST = ("boardetect_test", )
predictor = DefaultPredictor(cfg)

from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_board_dicts_test()
decresult = 0
for d in random.sample(dataset_dicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=board_metadata_test, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE   # remove the colors of unsegmented pixels
    )
    decresult = decresult + len(outputs['instances'])
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])
print(f'number of detected boxes is {decresult}')


In [ ]:
im = cv2.imread('/content/drive/MyDrive/barcode/capture_2018_08_08_05_30_52.png')
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=board_metadata_test, 
                scale=0.8,
                instance_mode=ColorMode.IMAGE   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
outputs["instances"].scores

tensor([0.9953, 0.9947, 0.9933, 0.9921, 0.9837, 0.9788, 0.9763, 0.9586],
       device='cuda:0')

time: 5.17 ms (started: 2022-05-23 04:36:27 +00:00)


In [ ]:
def calculateIoU(contour1, contour2):
    # Two separate contours trying to check intersection on
    contours = [contour1, contour2]

    # Create image filled with zeros the same size of original image
    blank = np.zeros([1944, 2592]) 

    # Copy each contour into its own image and fill it with '1'
    image1 = cv2.drawContours(blank.copy(), contours, 0, 1, thickness=cv2.FILLED)
    image2 = cv2.drawContours(blank.copy(), contours, 1, 1, thickness=cv2.FILLED)


    area1 = cv2.contourArea(contour1)
    area2 = cv2.contourArea(contour2)
    intersection = np.logical_and(image1, image2)
    contours, hierarchy = cv2.findContours(intersection.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return 0
    common_area = cv2.contourArea(contours[0])
    IoU = common_area/(area1+area2-common_area)
    # Use the logical AND operation on the two images
    # Since the two images had bitwise and applied to it,
    # there should be a '1' or 'True' where there was intersection
    # and a '0' or 'False' where it didnt intersect
    # Check if there was a '1' in the intersection
    return IoU


def truePositive(predicted_box, true_box):
    sum_IoU = 0
    hit_count = 0
    for conpred in predicted_box:
      hit = 0
      for contrue in true_box:
        IoU = calculateIoU(conpred, contrue)
        if IoU>0.6:
          hit =  1
          sum_IoU = sum_IoU+IoU
      if hit == 1:
        hit_count = hit_count + 1
    if hit_count == 0:
      return 0, None
    ave_IoU = sum_IoU/hit_count    
    return hit_count, ave_IoU

def collect_annotation(dataframe):
  true_box = []
  for i in range(len(dataframe)):
    rec1 = dataframe.iloc[i, :]
    contour = np.array([[rec1.X1,rec1.Y1],[rec1.X2,rec1.Y2],[rec1.X3,rec1.Y3], [rec1.X4,rec1.Y4]], dtype=np.int32)
    true_box.append(contour)
  return true_box


def collect_predicted(result):
  Object, Mask = result
  useful_index = []
  for i in range(len(Object)):
    score = Object[i]
    if score > 0.6:
      useful_index.append(i)

  predicted_box = []
  predicted_contour = []
  for index in useful_index:
    mg = Mask[index].cpu().numpy().astype(np.uint8)
    contours, hierarchy = cv2.findContours(mg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    predicted_box.append(box)
    predicted_contour.append(cnt)
  return predicted_box, predicted_contour

time: 60.5 ms (started: 2022-05-23 04:41:25 +00:00)


In [ ]:
test_df['Color'] = test_df['Color'].str.strip()
image_name = test_df.loc[:, 'Image Filename'].unique() # get unique item in one column 
img_address_base = '/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)/'
misimage = []

sum_hitR = 0
sum_hitG = 0
sum_hitB = 0

pred_total = 0

sum_R = 0
sum_G = 0
sum_B = 0
for i in tqdm(range(len(image_name))):
  df2 = df[df.loc[:, 'Image Filename']==image_name[i]] # choose data of one image 
  imgAddress = img_address_base + image_name[i] + '.jpg'  
  img = cv2.imread(imgAddress)
  output = predictor(img)
  result = (output["instances"].scores, output["instances"].pred_masks)
  predicted_box, predicted_contour = collect_predicted(result)
  dfR = df2[df2.loc[:, 'Color'] == 'R']
  dfG = df2[df2.loc[:, 'Color'] == 'G']
  dfB = df2[df2.loc[:, 'Color'] == 'B']
  true_box = collect_annotation(df2)
  true_boxR = collect_annotation(dfR)
  true_boxG = collect_annotation(dfG)
  true_boxB = collect_annotation(dfB)
  #hit, IoU = truePositive(predicted_box, true_box)
  hitR, IoR = truePositive(predicted_box, true_boxR)
  hitG, IoG = truePositive(predicted_box, true_boxG)
  hitB, IoB = truePositive(predicted_box, true_boxB)

  sum_hitR += hitR
  sum_hitG += hitG
  sum_hitB += hitB

  sum_R += len(true_boxR) 
  sum_G += len(true_boxG) 
  sum_B += len(true_boxB) 

  pred_total += len(predicted_box)

  sum_hit = hitR + hitB + hitG
  sum_all = len(true_boxR)  + len(true_boxG) + len(true_boxB) 
  if sum_hit!=sum_all:
    img = cv2.imread(imgAddress)
    cv2.drawContours(img, predicted_box, -1 ,(0,255,0),2) # predicted result is green
    cv2.drawContours(img, true_box, -1, (255,0,0), 2) # annotated result is blue 
    cv2.drawContours(img, predicted_contour, -1,(0,0,255),2) # the predicted contour is red
    cv2_imshow(img)
    print(f'number of R mishit is {len(true_boxR) - hitR}')
    print(f'number of G mishit is {len(true_boxG) - hitG}')
    print(f'number of B mishit is {len(true_boxB) - hitB}')

In [ ]:
# The accuracy of the algorithm is
(sum_hitR + sum_hitG + sum_hitB)/(sum_R + sum_G + sum_B)

0.9124087591240876

time: 3.91 ms (started: 2022-05-23 04:50:52 +00:00)


In [ ]:
TP_all = sum_hitR + sum_hitG + sum_hitB
TP_R = sum_hitR
TP_G = sum_hitG
TP_B = sum_hitB

FN_all = sum_R + sum_G + sum_B - sum_hitR - sum_hitG - sum_hitB 
FN_R = sum_R - sum_hitR
FN_G = sum_G - sum_hitG
FN_B = sum_B - sum_hitB

#since for a mis predicted, it can not be count into R or G or B. So we just assume all three of them get an average false Positive 
FP = pred_total - sum_hitR - sum_hitG - sum_hitB 

# TN can not be calcuated 

F1_all = 2*TP_all/(2*TP_all+FN_all+FP)
F1_R = 2*TP_R/(2*TP_R+FN_R+FP/3)
F1_G = 2*TP_G/(2*TP_G+FN_G+FP/3)
F1_B = 2*TP_B/(2*TP_B+FN_B+FP/3)

time: 6.02 ms (started: 2022-05-23 05:43:29 +00:00)


In [ ]:
print('Report for Detectron 2 model with mask_rcnn_R_50_FPN_3x')
print(f'for all data TP is {TP_all}, FN is {FN_all}, FP is {FP}, TN can not be calculated, the F1 score is {F1_all:.2f}')
print(f'for R data TP is {TP_R}, FN is {FN_R}, FP is {FP}, TN can not be calculated, the F1 score is {F1_R:.2f}')
print(f'for G data TP is {TP_G}, FN is {FN_G}, FP is {FP}, TN can not be calculated, the F1 score is {F1_G:.2f}')
print(f'for B data TP is {TP_B}, FN is {FN_B}, FP is {FP}, TN can not be calculated, the F1 score is {F1_B:.2f}')

for all data TP is 125, FN is 12, FP is 14, TN can not be calculated, the F1 score is 0.91
for R data TP is 77, FN is 2, FP is 14, TN can not be calculated, the F1 score is 0.96
for G data TP is 34, FN is 3, FP is 14, TN can not be calculated, the F1 score is 0.90
for B data TP is 14, FN is 7, FP is 14, TN can not be calculated, the F1 score is 0.71
time: 2.07 ms (started: 2022-05-23 05:43:31 +00:00)
